### Load LLM

In [21]:
%pip install --quiet -U langchain-ollama

Note: you may need to restart the kernel to use updated packages.


In [22]:
from langchain_ollama.chat_models import ChatOllama

local_llm = "llama3.2:1b-instruct-fp16"
llm = ChatOllama(
    base_url="http://localhost:11434",
    model = local_llm,
    # temperature = 0.8,
    # num_predict = 256,
    # format="json",
    # other params ...
)

# Test message
messages = [
    ("system", "You are a helpful translator. Translate the user sentence to French."),
    ("human", "I love programming."),
]
llm.invoke(messages)

AIMessage(content='Vous aimez programmmer.', additional_kwargs={}, response_metadata={'model': 'llama3.2:1b-instruct-fp16', 'created_at': '2025-04-15T03:47:06.4255961Z', 'done': True, 'done_reason': 'stop', 'total_duration': 871054300, 'load_duration': 16841300, 'prompt_eval_count': 42, 'prompt_eval_duration': 595000000, 'eval_count': 7, 'eval_duration': 257000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-880a5e6d-bbeb-4240-80a5-2d2e9a23909a-0', usage_metadata={'input_tokens': 42, 'output_tokens': 7, 'total_tokens': 49})

## Stateless Nature of LLMs

In [23]:
from langchain_core.messages import HumanMessage

llm.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content="Hello Bob! How's it going?", additional_kwargs={}, response_metadata={'model': 'llama3.2:1b-instruct-fp16', 'created_at': '2025-04-15T03:47:06.9651073Z', 'done': True, 'done_reason': 'stop', 'total_duration': 529604100, 'load_duration': 17111200, 'prompt_eval_count': 30, 'prompt_eval_duration': 175000000, 'eval_count': 9, 'eval_duration': 337000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-1c3db5d8-fbf9-4e31-ab14-959390809223-0', usage_metadata={'input_tokens': 30, 'output_tokens': 9, 'total_tokens': 39})

In [24]:
llm.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="I don't have any information about you, so I'm not aware of your name. This is the beginning of our conversation, and I'm here to help with any questions or topics you'd like to discuss. Is there something specific you'd like to talk about or ask about?", additional_kwargs={}, response_metadata={'model': 'llama3.2:1b-instruct-fp16', 'created_at': '2025-04-15T03:47:09.6692977Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2694440400, 'load_duration': 12655400, 'prompt_eval_count': 30, 'prompt_eval_duration': 166000000, 'eval_count': 58, 'eval_duration': 2515000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-9d2f7c27-6a8f-471a-ac49-93c236170241-0', usage_metadata={'input_tokens': 30, 'output_tokens': 58, 'total_tokens': 88})

### Multi-turn Conversation

In [25]:
from langchain_core.messages import AIMessage

llm.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content="I don't have any information about you, Bob. I'm a large language model, I don't have the ability to store or recall personal data about individual users. Each time you interact with me, it's a new conversation and I don't retain any information from previous conversations. Is there something else I can help you with?", additional_kwargs={}, response_metadata={'model': 'llama3.2:1b-instruct-fp16', 'created_at': '2025-04-15T03:47:13.1318953Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3452264300, 'load_duration': 12995300, 'prompt_eval_count': 55, 'prompt_eval_duration': 563000000, 'eval_count': 68, 'eval_duration': 2874000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-6b6fe42a-2241-4b86-9e00-c09660281a8c-0', usage_metadata={'input_tokens': 55, 'output_tokens': 68, 'total_tokens': 123})

In [26]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = llm.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [27]:
config = {"configurable": {"thread_id": "abc123"}}

In [28]:
query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

Hello Bob! How's it going?


In [29]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I made a mistake by not addressing you by your name earlier. You asked me to call you "Bob", and I should have done that. Let's start fresh! What's on your mind, Bob?


In [30]:
output["messages"]

[HumanMessage(content="Hi! I'm Bob.", additional_kwargs={}, response_metadata={}, id='ca4bc3dd-e723-421a-8722-6d2ea511a256'),
 AIMessage(content="Hello Bob! How's it going?", additional_kwargs={}, response_metadata={'model': 'llama3.2:1b-instruct-fp16', 'created_at': '2025-04-15T03:47:13.6923631Z', 'done': True, 'done_reason': 'stop', 'total_duration': 521435100, 'load_duration': 13125200, 'prompt_eval_count': 31, 'prompt_eval_duration': 165000000, 'eval_count': 9, 'eval_duration': 342000000, 'message': {'role': 'assistant', 'content': '', 'images': None, 'tool_calls': None}}, id='run-9ba91e06-2a53-4291-90fc-8efe8d51e3bc-0', usage_metadata={'input_tokens': 31, 'output_tokens': 9, 'total_tokens': 40}),
 HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}, id='0f73a052-a89b-44bd-a175-d79f68bb48e4'),
 AIMessage(content='I made a mistake by not addressing you by your name earlier. You asked me to call you "Bob", and I should have done that. Let\'s start fresh